In [1]:
# Import the modules
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.metrics import balanced_accuracy_score, confusion_matrix, classification_report
from sklearn.preprocessing import StandardScaler

---

## Split the Data into Training and Testing Sets

### Step 1: Read the `lending_data.csv` data from the `Resources` folder into a Pandas DataFrame.

In [2]:
# Read the CSV file from the Resources folder into a Pandas DataFrame
df_lending = pd.read_csv("Resources/lending_data.csv")

# Review the DataFrame
df_lending.sample(10)

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,loan_status
37469,9300.0,7.070,47100,0.363057,3,0,17100,0
33849,8000.0,6.526,42000,0.285714,2,0,12000,0
30114,11100.0,7.834,54300,0.447514,5,1,24300,0
13276,10400.0,7.546,51600,0.418605,4,1,21600,0
22928,8800.0,6.877,45300,0.337748,3,0,15300,0
46923,9700.0,7.261,48900,0.386503,4,0,18900,0
25831,8400.0,6.688,43500,0.310345,3,0,13500,0
41192,8700.0,6.817,44700,0.328859,3,0,14700,0
24220,8600.0,6.771,44300,0.322799,3,0,14300,0
23540,11100.0,7.856,54500,0.449541,5,1,24500,0


In [3]:
df_lending.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77536 entries, 0 to 77535
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   loan_size         77536 non-null  float64
 1   interest_rate     77536 non-null  float64
 2   borrower_income   77536 non-null  int64  
 3   debt_to_income    77536 non-null  float64
 4   num_of_accounts   77536 non-null  int64  
 5   derogatory_marks  77536 non-null  int64  
 6   total_debt        77536 non-null  int64  
 7   loan_status       77536 non-null  int64  
dtypes: float64(3), int64(5)
memory usage: 4.7 MB


In [4]:
df_lending.describe()

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,loan_status
count,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000
mean,9805.562577,7.292333,49221.949804,0.377318,3.826610,0.392308,19221.949804,0.032243
std,2093.223153,0.889495,8371.635077,0.081519,1.904426,0.582086,8371.635077,0.176646
min,5000.000000,5.250000,30000.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,8700.000000,6.825000,44800.000000,0.330357,3.000000,0.000000,14800.000000,0.000000
50%,9500.000000,7.172000,48100.000000,0.376299,4.000000,0.000000,18100.000000,0.000000
75%,10400.000000,7.528000,51400.000000,0.416342,4.000000,1.000000,21400.000000,0.000000
max,23800.000000,13.235000,105200.000000,0.714829,16.000000,3.000000,75200.000000,1.000000


### Step 2: Create the labels set (`y`)  from the “loan_status” column, and then create the features (`X`) DataFrame from the remaining columns.

In [5]:
# Separate the data into labels and features

# Separate the y variable, the labels
y = df_lending['loan_status']

# Separate the X variable, the features
X = df_lending[df_lending.columns[:-1]]

In [6]:
# Review the y variable Series
y

0        0
1        0
2        0
3        0
4        0
        ..
77531    1
77532    1
77533    1
77534    1
77535    1
Name: loan_status, Length: 77536, dtype: int64

In [7]:
# Review the X variable DataFrame
X.sample(10)

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt
57515,9000.0,6.930,45800,0.344978,3,0,15800
48402,10400.0,7.555,51700,0.419729,4,1,21700
7499,11900.0,8.197,57700,0.480069,6,1,27700
18524,8800.0,6.844,45000,0.333333,3,0,15000
53894,10200.0,7.444,50700,0.408284,4,1,20700
68710,9600.0,7.185,48200,0.377593,4,0,18200
43845,9900.0,7.326,49500,0.393939,4,0,19500
44701,10600.0,7.642,52500,0.428571,5,1,22500
49271,7800.0,6.437,41200,0.271845,2,0,11200
68575,10200.0,7.472,50900,0.410609,4,1,20900


### Step 3: Check the balance of the labels variable (`y`) by using the `value_counts` function.

In [8]:
# Check the balance of our target values
unique_value_count = df_lending['loan_status'].value_counts()
unique_value_count

0    75036
1     2500
Name: loan_status, dtype: int64

In [9]:
print(f'Proportion of value  1 in dependent variable is {round((unique_value_count[1])*100/df_lending.shape[0], 2)}%')

Proportion of value  1 in dependent variable is 3.22%


##### Dependent variable `Loan_Status` seems to be imbalanced with  majority of the values being 0 and only 3.22% of the values are 1

### Step 4: Split the data into training and testing datasets by using `train_test_split`.

In [10]:
# Import the train_test_learn module
from sklearn.model_selection import train_test_split

# Split the data using train_test_split
# Assign a random_state of 1 to the function

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, stratify=y, random_state = 1)

### Step 5: Scaling input data using `StandardScaler`

In [11]:
scaler = StandardScaler()
X_scaler= scaler.fit(X_train)
X_scaler_train = X_scaler.transform(X_train)
X_scaler_test = X_scaler.transform(X_test)
pd.DataFrame(X_scaler_train, columns = X_train.columns).describe()

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt
count,5.427500e+04,5.427500e+04,5.427500e+04,5.427500e+04,5.427500e+04,5.427500e+04,5.427500e+04
mean,-2.283694e-16,-2.843265e-16,-1.918175e-16,1.446338e-16,-1.423714e-16,-1.369538e-16,-1.918175e-16
std,1.000009e+00,1.000009e+00,1.000009e+00,1.000009e+00,1.000009e+00,1.000009e+00,1.000009e+00
min,-2.292847e+00,-2.293201e+00,-2.293227e+00,-4.626126e+00,-2.006720e+00,-6.747843e-01,-2.293227e+00
25%,-5.285491e-01,-5.257745e-01,-5.285785e-01,-5.768788e-01,-4.343226e-01,-6.747843e-01,-5.285785e-01
50%,-1.470794e-01,-1.363796e-01,-1.351097e-01,-1.375652e-02,8.980991e-02,-6.747843e-01,-1.351097e-01
75%,2.820741e-01,2.653592e-01,2.702824e-01,4.909495e-01,8.980991e-02,1.042794e+00,2.702824e-01
max,6.528642e+00,6.545053e+00,6.541937e+00,4.098726e+00,6.379400e+00,4.477950e+00,6.541937e+00


---

## Create a Logistic Regression Model with the Original Data

###  Step 1: Fit a logistic regression model by using the training data (`X_train` and `y_train`).

In [12]:
# Import the LogisticRegression module from SKLearn
from sklearn.linear_model import LogisticRegression

# Instantiate the Logistic Regression model
# Assign a random_state parameter of 1 to the model
model =  LogisticRegression(random_state = 1)
# Fit the model using training data
model.fit(X_scaler_train, y_train)

LogisticRegression(random_state=1)

### Step 2: Save the predictions on the testing data labels by using the testing feature data (`X_test`) and the fitted model.

In [13]:
# Make a prediction using the testing data
y_pred = model.predict(X_scaler_test)

### Step 3: Evaluate the model’s performance by doing the following:

* Calculate the accuracy score of the model.

* Generate a confusion matrix.

* Print the classification report.

In [14]:
# Print the balanced_accuracy score of the model
balanced_acc =  balanced_accuracy_score(y_test, y_pred)
print(balanced_acc)

0.9888456606399834


In [15]:
# Generate a confusion matrix for the model
matrix = confusion_matrix(y_test, y_pred)
pd.DataFrame(matrix, index = ['Actual 0', 'Actual 1'], columns = ['Pred 0', 'Pred 1'])

,Pred 0,Pred 1
Actual 0,22399,112
Actual 1,13,737


In [16]:
# Print the classification report for the model
report = classification_report(y_test, y_pred)
print(report)

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     22511
           1       0.87      0.98      0.92       750

    accuracy                           0.99     23261
   macro avg       0.93      0.99      0.96     23261
weighted avg       1.00      0.99      0.99     23261



### Step 4: Answer the following question.

**Question:** How well does the logistic regression model predict both the `0` (healthy loan) and `1` (high-risk loan) labels?

**Answer:** Overall balanced accuracy is close to 99%. Recall of class `0` is almost 100% while that of class `1` is 92%. So we can conclude that model did a very good job of predicting the classes on the test data

---

## Predict a Logistic Regression Model with Resampled Training Data

### Step 1: Use the `RandomOverSampler` module from the imbalanced-learn library to resample the data. Be sure to confirm that the labels have an equal number of data points. 

In [17]:
# Import the RandomOverSampler module form imbalanced-learn
from imblearn.over_sampling import RandomOverSampler

# Instantiate the random oversampler model
# # Assign a random_state parameter of 1 to the model
ros_model = RandomOverSampler(random_state=1)

# Fit the original training data to the random_oversampler model
X_resampled, y_resampled =  ros_model.fit_resample(X, y)

In [18]:
# Count the distinct values of the resampled labels data
y_resampled.value_counts()

0    75036
1    75036
Name: loan_status, dtype: int64

### Step 2: Use the `LogisticRegression` classifier and the resampled data to fit the model and make predictions.

In [19]:
# splitting resampled data into train and test
X_resample_train, X_resample_test, y_resample_train, y_resample_test = train_test_split(X_resampled, y_resampled, 
                                                                                        test_size = 0.3, stratify=y_resampled,
                                                                                      random_state=1)


In [20]:
# Scaling the resampled data
X_resample_scaler = scaler.fit(X_resample_train)
X_resample_scaler_train = X_resample_scaler.transform(X_resample_train)
X_resample_scaler_test = X_resample_scaler.transform(X_resample_test)
pd.DataFrame(X_resample_scaler_train, columns = X_resample_train.columns).describe()

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt
count,1.050500e+05,1.050500e+05,1.050500e+05,1.050500e+05,1.050500e+05,1.050500e+05,1.050500e+05
mean,3.862794e-18,-4.873356e-17,-2.170985e-17,3.941244e-16,1.019418e-16,-8.632335e-16,-2.170985e-17
std,1.000005e+00,1.000005e+00,1.000005e+00,1.000005e+00,1.000005e+00,1.000005e+00,1.000005e+00
min,-1.900110e+00,-1.900173e+00,-1.900188e+00,-3.461690e+00,-1.779480e+00,-1.220560e+00,-1.900188e+00
25%,-9.520848e-01,-9.597583e-01,-9.573392e-01,-8.985186e-01,-8.447578e-01,-1.220560e+00,-9.573392e-01
50%,-2.512685e-02,-1.983952e-02,-1.975766e-02,2.653406e-01,8.996421e-02,-2.410947e-01,-1.975766e-02
75%,9.650328e-01,9.681658e-01,9.704970e-01,9.621549e-01,1.024686e+00,7.383701e-01,9.704970e-01
max,1.997327e+00,2.004258e+00,2.002890e+00,1.420645e+00,1.959408e+00,1.717835e+00,2.002890e+00


In [21]:
# Instantiate the Logistic Regression model
# Assign a random_state parameter of 1 to the model
model_resample =  LogisticRegression(random_state = 1)

# Fit the model using the resampled training data
model_resample.fit(X_resample_scaler_train, y_resample_train)

# Make a prediction using the testing data
y_resample_pred = model_resample.predict(X_resample_scaler_test)

### Step 3: Evaluate the model’s performance by doing the following:

* Calculate the accuracy score of the model.

* Generate a confusion matrix.

* Print the classification report.

In [22]:
# Print the balanced_accuracy score of the model 
balanced_acc_resample =  balanced_accuracy_score(y_resample_test, y_resample_pred)
print(balanced_acc_resample)

0.9947359068899649


In [23]:
# Generate a confusion matrix for the model
matrix_resample = confusion_matrix(y_resample_test, y_resample_pred)
pd.DataFrame(matrix_resample, index = ['Actual 0', 'Actual 1'], columns = ['Pred 0', 'Pred 1'])

,Pred 0,Pred 1
Actual 0,22389,122
Actual 1,115,22396


In [24]:
# Print the classification report for the model
report_resample = classification_report(y_resample_test, y_resample_pred)
print(report_resample)

              precision    recall  f1-score   support

           0       0.99      0.99      0.99     22511
           1       0.99      0.99      0.99     22511

    accuracy                           0.99     45022
   macro avg       0.99      0.99      0.99     45022
weighted avg       0.99      0.99      0.99     45022



### Step 4: Answer the following question

**Question:** How well does the logistic regression model, fit with oversampled data, predict both the `0` (healthy loan) and `1` (high-risk loan) labels?

**Answer:** Model with oversampled data did comparatively better than original data. Total balanced accuracy further improved from 98.88% to 99.47%. There is an improvement in `Precision` and `Recall` of class `1` , while a slight deterioration in prediction accuracy for class `0`. As a business manager, I would trade off improvement in `Recall` for class `1` high risk loan with slight decline in performance for class`0`